In [ ]:
# MIT License

# Copyright (c) 2023 Looker Data Sciences, Inc.

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Explore Assistant Examples Generation

This notebook is a companion to the [Explore Assistant Looker + GenAI Solution](https://github.com/LukaFontanilla/looker-explore-assistant) and will take you through some example code for:


*   Formatting Looker Explore Metadata for Prompt Stuffing
*   Generating NLQ to Explore URL examples from your data and Looker Explore usage
*   Categorizing those examples by different Looker query types



## Install Dependencies

In [1]:
%pip install looker-sdk
%pip install --upgrade google-cloud-aiplatform

  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 kB 4.5 MB/s eta 0:00:000m eta 0:00:01
Using cached attrs-23.2.0-py3-none-any.whl (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 806.4 kB/s eta 0:00:00 eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached google_cloud_resource_manager-1.12.3-py2.py3-none-any.whl.metadata (5.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 

## Import Required Packages

In [2]:
import looker_sdk
import vertexai
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig
from looker_sdk import models40 as models, error
import configparser
import json
import urllib.parse
import re

## Configure Application Default Credentials with GCloud

This [Exports ADC credentials](https://cloud.google.com/docs/authentication/application-default-credentials) to your environment. ***Make sure to set the quota project to a GCP project that has the Vertex AI API enabled.***

In [3]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=CWojzIy4VaGkBHBs529aWnSP5xSfpc&access_type=offline&code_challenge=BjJytUqYFUaDuRuqHbhraygzCBBUwMP2a2lnt_sFkA0&code_challenge_method=S256


Credentials saved to file: [/Users/quentinchuret/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "m33-datawarehouse-silver" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some G

In [4]:
!gcloud config set project looker-private-demo
!gcloud auth application-default set-quota-project looker-private-demo


To update your Application Default Credentials quota project, use the `gcloud auth application-default set-quota-project` command.
Are you sure you wish to set property [core/project] to looker-private-demo?

Do you want to continue (Y/n)?  ^C


Command killed by keyboard interrupt

ERROR: (gcloud.auth.application-default.set-quota-project) User [quentinch@theodo.fr] does not have permission to access projects instance [looker-private-demo:testIamPermissions] (or it may not exist): Caller does not have required permission to use project looker-private-demo. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=looker-private-demo and then retry. Propagation of the new permission may take a few minutes.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developer console IAM admin
    url: https://conso

## Setup Looker SDK

Steps for configuring the Looker Python SDK:


1.   Create a file named `looker.ini`
2.   Using the example below, fill in the variables as they are for your environment. ***You will need Looker API credentials for a user that has at least `explore` level permissions.***
2.   Upload that file into your Colab Notebook


`looker_example.ini`
```
[Looker]
base_url=
client_id=
client_secret=
verify_ssl=true
```





In [10]:
sdk = looker_sdk.init40('looker.ini')

## Setup Vertex Python SDK with Gemini Model

Please set the following variables prior to running the cell:



In [36]:
project_id = 'm33-datawarehouse-silver' # @param {type:"string"}
location = 'us-central1' # @param {type:"string"}

prompt = '''You are a specialized assistant that translates Looker Explore query URL's into natural language questions. By reading the different parameters of the url (like the fields used, filters, etc.) you are able to generate a natural language question.
Please keep these responses short and concise using 1-2 sentences max in your repsonse. Make sure to generate a response that sounds like it's coming from an average person and not a data analyst who is very familiar with the data. Each request will contain an "input" and an "output" field. The "output" field will be the Looker Explore query url. The "input" field will be the natural language question that you will fill in/generate. Here is an example of a properly formatted response:
Example:
{"input": "customer with lifetime revenue > 100", "output": "fields=user_order_facts.lifetime_revenue&f[user_order_facts.lifetime_revenue]=>100&sorts=user_order_facts.lifetime_revenue desc 0&limit=500"}

Here is the output of the request. Please generate a natural language question based on the output. Just give me the natural language question that you would ask based on the output.
'''

parameters = {"max_output_tokens": 2500, "temperature": 0.2, "candidate_count": 1}
vertexai.init(project=project_id, location=location)

def generate_input(request):
    model = GenerativeModel("gemini-pro")
    # make prediction to generate Looker Explore URL
    response =  model.generate_content(
        contents=prompt + request,
        generation_config=GenerationConfig(
            temperature=0.2,
            top_p=0.8,
            top_k=40,
            max_output_tokens=1000,
            candidate_count=1
        )
    )

    return response.text

## Format Looker Explore Metadata for Prompt Stuffing

The next two cells provide the Looker Explore Metadata the LLM needs to be able to generate Looker Explore URL's from natural language. This is all done through Prompt Stuffing.


*   Fetches all the field metadata from the LookML model for a given Explore
*   Generates two arrays containing all the measures and dimensions with the name, type, description and any other relevant attribute you'd like to include
*   Formats that into a structured text variable and writes it to a txt file in the format of `model::explore.txt`





In [26]:
def fetchExploreMetadata(model,explore,fields):
  data = sdk.lookml_model_explore(model,explore,fields)

  # Dimensions
  dimensions = []
  for field in data.fields.dimensions:
    dimensions.append(f"name: {field.name}, type: {field.type}, description: {field.description} \n")

  # # Measures
  measures = []
  for field in data.fields.measures:
    measures.append(f"name: {field.name}, type: {field.type}, description: {field.description} \n")

  return {
      "dimensions": dimensions,
      "measures": measures
  }


def formatExploreMetadata(data):
  return f"""
  Dimensions Used to group by information:\n {''.join(data['dimensions'])}
  Measures are used to perform calculations/aggregations (if top, bottom, total, sum, etc. are used include a measure):\n {''.join(data['measures'])}
  """


In [27]:
model = 'sales' # @param {type:"string"}
explore = 'deals' # @param {type:"string"}

data = fetchExploreMetadata(model, explore, 'fields')

with open(f"./{model}::{explore}.txt", "w") as f:
  f.write(formatExploreMetadata(data))

## Setup Explore URL Parser & Categorizer

The following cells setup the functions used to generate commmon and representative Looker Explore query URL's that are labeled via Gen AI for the LLM to use in it's reasoning.

*   CONSTANTS: Regex patterns for Looker Query filter string parsing
*   LOOKER QUERY METHODS: The functions for fetching historical queries and parsing their metadata for an expanded URL
*   LOOKER URL PARSER FUNCTIONS: Functions used to parse and categorize example URL's into specific categories of queries





In [28]:
### CONSTANTS

# Time / Date Regex Patterns
time_relative_pattern = r"(\d+)\s+(month|week|day|year)?(?:\s+ago)?"
time_range_pattern = r"\b(\d+)\s+(month|week|day|year)s?\s+ago\s+for\s+\1\s+\2\b"

# Numerical Patterns
numerical_comparison_pattern = r"^(>|>=|<|<=|<>)?(\d+)$"
numerical_range_pattern = r"\b(>|>=|<|<=|<>)?(\d+)?\s+(AND|OR)?\s+(>|>=|<|<=|<>)?(\d+)"

# String Patterns
string_catch_all_pattern = r"\w"
string_multiple_pattern = r"\w,+\w"
categorized_queries = {}
categorized_queries_filters = {}

### END


### LOOKER QUERY METHODS


def fetchQueryUrlMetadata(explore: str):
  try:
    response = sdk.run_inline_query(
        result_format='json',
        cache=True,
        body=models.WriteQuery(
            model="system__activity",
            view="history",
            fields=[
              "query.slug",
              "query.view",
              "query.dynamic_fields",
              "query.formatted_fields",
              "query.filters",
              "query.filter_expression",
              "query.formatted_pivots",
              "query.sorts",
              "query.limit",
              "query.column_limit",
              "query.count"
            ],
            pivots=None,
            fill_fields=None,
            filters={
              "query.view": explore,
              "history.status": "complete",
              # "user.email":""
            },
            filter_expression=None,
            sorts=[
              "history.completed_time desc"
              "query.view"
            ],
            limit="10",
        )
    )

    return json.loads(response)[0:10]
  except error.SDKError as e:
    print(e.message)



def fetchQueryUrl(slug: str):
  query_url = sdk.query_for_slug(slug=slug)
  return query_url

### END


### LOOKER URL PARSER FUNCTIONS

# limit categorization
def limit_categorization(query,url):
  if "query.limit" in query and query['query.limit'] != None:
      categorized_queries.setdefault('limit',[])
      categorized_queries['limit'].append(url)

# dynamic fields categorization
def dynamic_fields_categorization(query,url):
  if "query.dynamic_fields" in query and query['query.dynamic_fields'] != None:
      categorized_queries.setdefault('dynamic_fields',[])
      categorized_queries['dynamic_fields'].append(url)

# sorts categorization
def sorts_categorization(query,url):
  if "query.sorts" in query and query['query.sorts'] != None:
      categorized_queries.setdefault('sorts',[])
      categorized_queries['sorts'].append(url)

# filter expression categorization
def filter_expression_categorization(query,url):
  if "query.filter_expression" in query and query['query.filter_expression'] != None:
      categorized_queries.setdefault('filter_expression',[])
      categorized_queries['filter_expression'].append(url)

# pivots categorization
def pivots_categorization(query,url):
  if "query.formatted_pivots" in query and query['query.formatted_pivots'] != None:
      categorized_queries.setdefault('pivots',[])
      categorized_queries['pivots'].append(url)

# filters categorization
def filters_categorization(query,url):
  parsed_filters = json.loads(query['query.filters'])
  keys_copy = tuple(parsed_filters.keys())
  for key in keys_copy:
    if parsed_filters[key] != "":
      if re.findall(time_range_pattern, parsed_filters[key]):
        categorized_queries_filters.setdefault('time_range',[])
        categorized_queries_filters['time_range'].append(url)
        continue
      if re.findall(time_relative_pattern, parsed_filters[key]):
        categorized_queries_filters.setdefault('time_relative',[])
        categorized_queries_filters['time_relative'].append(url)
        continue
      elif re.findall(numerical_comparison_pattern, parsed_filters[key]):
        categorized_queries_filters.setdefault('numerical_comparison',[])
        categorized_queries_filters['numerical_comparison'].append(url)
        continue
      elif re.findall(numerical_range_pattern, parsed_filters[key]):
        categorized_queries_filters.setdefault('numerical_range',[])
        categorized_queries_filters['numerical_range'].append(url)
        continue
      elif re.findall(string_multiple_pattern, parsed_filters[key]):
        categorized_queries_filters.setdefault('string_multiple',[])
        categorized_queries_filters['string_multiple'].append(url)
        continue
      elif re.findall(r"\w",parsed_filters[key]):
        categorized_queries_filters.setdefault('string_standard',[])
        categorized_queries_filters['string_standard'].append(url)
        continue

### END

def explore_url_categorization(data):
  for query in data:
    query_data = fetchQueryUrl(str(query['query.slug']))
    decoded_url = urllib.parse.unquote(query_data.url)

    # return url parameters as a string
    url_parameters = decoded_url.split("?",1)[1].replace("+", " ")
    # remove timezone parameter
    decoded_url_notimezone = re.sub(r"&query_timezone=(.)*&", "&", url_parameters,count=1)
    # remove filter config parameter
    decoded_url_nofilterconfig = re.sub(r"&filter_config=(.)*(?=&|$)", "&", decoded_url_notimezone)[0:-1] if re.sub(r"&filter_config=(.)*(?=&|$)", "&", decoded_url_notimezone)[-1] == "&" else re.sub(r"&filter_config=(.)*(?=&|$)", "&", decoded_url_notimezone)
    # parse vis config parameter only maintain vis type
    vis_config = re.search(r"(&vis=(.)*(?=&|$))", decoded_url_nofilterconfig)

    decoded_url_modifiedvisjson = ''
    if vis_config:
      vis_json_str = vis_config.group(1)
      # regex to search for the vis type (ie. "type":"looker_bar")
      vis_type = re.search(r'("type":\s*"([^,}]+))', vis_json_str)
      # replace the vis config in original url parameter string with the modified vis type
      decoded_url_modifiedvisjson = re.sub(r"(&vis=(.)*(?=&|$))","&vis={" + (vis_type.group(1) if vis_type else '') + "}",decoded_url_nofilterconfig)
    else:
      decoded_url_modifiedvisjson = decoded_url_nofilterconfig

    # run categorization functions to construct object with categorized urls
    limit_categorization(query,decoded_url_modifiedvisjson)
    dynamic_fields_categorization(query,decoded_url_modifiedvisjson)
    sorts_categorization(query,decoded_url_modifiedvisjson)
    filter_expression_categorization(query,decoded_url_modifiedvisjson)
    pivots_categorization(query,decoded_url_modifiedvisjson)
    filters_categorization(query,decoded_url_modifiedvisjson)

  categorized_queries.setdefault('filters',{})
  categorized_queries['filters'] = categorized_queries_filters
  return categorized_queries


In [33]:
data = fetchQueryUrlMetadata('deals')
# categorization =
categorized_queries = explore_url_categorization(data)

print(categorized_queries)

{'limit': ['fields=offers.offer_title,deal_owners.owner_name,deals.deal_name,companies.name,deals.partner,deals.stage_name,deals.hot_mild,deals.cloud_provider_of_the_project,offers_to_deals.offer_amount_in_euro,deals.close_date,deals.days_lead_time&f[deals.is_won]=No&f[deals.stage_name]=-Lead KO,-Lost&f[offers.offer_title]=Sicara - Data ESG&sorts=deals.close_date desc&limit=500&column_limit=50&vis={"type":"looker_grid"}', 'fields=offers.offer_title,offers.owner,deals.deal_name,deals_to_companies.list_of_companies,deal_owners.owner_name,offers_to_deals.offer_amount_in_euro&f[deals.is_won]=Yes&f[offers.is_wbr]=Yes&f[deals.startup]=Hokla,Theodo FR&f[deals.close_date]=1 week ago for 1 week&f[companies.name]=&sorts=deals_to_companies.list_of_companies desc 0&limit=500&column_limit=50&vis={"type":"looker_grid"}', 'fields=companies.name,deals.deal_name,deal_owners.owner_name,deals.days_lead_time,deals_to_companies.is_new_business,deals.total_amount,contacts.contact_list,offers_to_deals.list_o

In [42]:
url_prompts = []

for key in categorized_queries.keys():
  if type(categorized_queries[key]) == list:
    for url in categorized_queries[key][0:3]:
      input = generate_input(url)
      print(input)
      url_prompts.append(json.dumps({"input": input, "output": url}) + '\n')

  else:
    for key2 in categorized_queries[key].keys():
      for url in categorized_queries[key][key2][0:3]:
        input = generate_input(url)
        print(input)
        url_prompts.append(json.dumps({"input": input, "output": url}) + '\n')


with open("./examples.jsonl", "a") as f:
  f.writelines(url_prompts)

What are the names of the deals that were lost or in the Lead KO stage, and which companies were involved? 

What are the names of the offers, owners, deals, companies, and deal owners for deals won by Hokla and Theodo FR in the past week, along with the offer amount in euros? 

What are the names of the companies, deals, and deal owners, along with the lead time, new business status, total amount, contact list, list of offers, and client category, for deals in stage Column B, for startups Theodo FR and Hokla, excluding Bpifrance, sorted by total amount in descending order, with a limit of 500 results? 

Which deals were lost or in the Lead KO stage that included the Sicara - Data ESG offer? 

What are the names of the companies, deals, and deal owners, along with the lead time, new business status, total amount, contact list, list of offers, and client category, for deals in stage Column B, for startups Theodo FR and Hokla, excluding Bpifrance, sorted by total amount descending, with 